In [1]:
import requests
from datetime import datetime
import pandas as pd
import time
import ccxt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

In [2]:
symbol = pd.read_excel('usdt_futures_symbol.xlsx')
symbol_list = symbol['EOSUSDT'].tolist()
symbol_list.append('EOSUSDT')

In [7]:
def to_datetime(timestamp) :
    return datetime.fromtimestamp(timestamp/1000)

def to_timestamp(date) :
    tmp = datetime.strptime(date, '%Y-%m-%d')
    return int((str(time.mktime(tmp.timetuple()))[:-2] + '000'))

In [5]:
def crawle_data(symbol) :
    
    try : 
        starting = time.time()

        ep='https://api.binance.com'
        candle = '/api/v3/klines'

        start_date = []
        start_time = []
        openn = []
        high = []
        low = []
        close = []
        volume = []

        #start = 1504224000000 # 17년 9월 1일 09:00:00 최초날짜
        start = to_timestamp('2020-09-01') # 테스트를 위해 최근만 받아옴 (20년 11월 )
        first_params_candle = {'symbol': symbol, 'interval' : '1m','startTime' : start , 'limit' : 1 } # 초기세팅
        r1 = requests.get(ep+candle, params=first_params_candle )

        while len(r1.json()) > 0 :
            first_params_candle = {'symbol': symbol, 'interval' : '1m','startTime' : start , 'limit' : 1000 }
            r1 = requests.get(ep+candle, params=first_params_candle ) #use parameter
            req = r1.json()
            for i in range(0,len(req)) : 
                start_date.append(datetime.fromtimestamp(req[i][0]/1000).strftime('%Y%m%d'))
                start_time.append(datetime.fromtimestamp(req[i][0]/1000).strftime('%H%M'))
                close.append(req[i][4])
            if len(req) > 0 : # 받아온 데이터가 있을 경우 => 다음 루프 시작
                start = req[-1][6]+1
            else : print(symbol + " 다운로드 완료")
            #time.sleep(0.25) # 횟수제한을 위해 추가

        ticker = [symbol] * len(start_date)

        chartData = {'date': start_date, 'time': start_time, 'ticker': ticker, 'close': close}

        df = pd.DataFrame(chartData, columns=['date', 'time', 'ticker', 'close'])
        df2 = df[['date', 'time', 'close']]
        df2.columns = ['date', 'time', symbol]

        print("소요 시간 : ", time.time() - starting)
        
        return df2
    
    # 데이터 안불러 와지는 경우 keyerror가 나므로 예외처리함
    except KeyError as e :
        print(e)

In [8]:
first_future = crawle_data('ETCUSDT')
first_future = first_future.set_index('date')

symbol_list2 = symbol_list.copy()
symbol_list2.remove('ETCUSDT')

ETCUSDT 다운로드 완료
소요 시간 :  28.4094979763031


In [10]:
for symbol in symbol_list2 : 
    
    try : 
        tmp = crawle_data(symbol)
        first_future[symbol] = tmp.iloc[:len(first_future), 2].tolist()
    
    # 데이터 없을 경우 tmp가 none type이 되므로 attributeerror가 나게 됨
    except AttributeError as e:
        print(e)
    
    except ConnectionError as e:
        print(e)
        
    except ValueError as e:
        print(e)
        
    except TimeoutError as e:
        print(e)

SUSHIUSDT 다운로드 완료
소요 시간 :  26.590320825576782
Length of values does not match length of index
CVCUSDT 다운로드 완료
소요 시간 :  24.777660608291626
BALUSDT 다운로드 완료
소요 시간 :  24.135157823562622
KNCUSDT 다운로드 완료
소요 시간 :  24.174421310424805
SRMUSDT 다운로드 완료
소요 시간 :  23.893764972686768
ENJUSDT 다운로드 완료
소요 시간 :  24.502938508987427
ZRXUSDT 다운로드 완료
소요 시간 :  23.99054217338562
QTUMUSDT 다운로드 완료
소요 시간 :  24.505005359649658
ZENUSDT 다운로드 완료
소요 시간 :  23.49861741065979
ATOMUSDT 다운로드 완료
소요 시간 :  25.227680921554565
IOTAUSDT 다운로드 완료
소요 시간 :  24.756871938705444
WAVESUSDT 다운로드 완료
소요 시간 :  24.317835807800293
ADAUSDT 다운로드 완료
소요 시간 :  25.555025100708008
NEARUSDT 다운로드 완료
소요 시간 :  16.01710557937622
Length of values does not match length of index
BCHUSDT 다운로드 완료
소요 시간 :  25.17302441596985
XTZUSDT 다운로드 완료
소요 시간 :  25.378799200057983
BNBUSDT 다운로드 완료
소요 시간 :  26.401140689849854
IOSTUSDT 다운로드 완료
소요 시간 :  24.979830026626587
HNTUSDT 다운로드 완료
소요 시간 :  20.54502272605896
Length of values does not match length of index
XMRUSDT 다운로드 완료


In [11]:
first_future.to_csv('future_close_tmp.csv')

In [12]:
future_close = pd.read_csv('future_close_tmp.csv')

In [15]:
# 데이터 날짜 전처리
future_close = first_future.reset_index()
future_close['Date'] = future_close['date'] + future_close['time']
future_close['Date'] = future_close['Date'].map(lambda x : datetime.strptime(x, '%Y%m%d%H%M'))
future_close.set_index('Date', inplace = True)
future_close = future_close.iloc[:,2:]
future_close = future_close.apply(pd.to_numeric)
future_list = future_close.columns.tolist()

In [16]:
future_close.to_csv('future_close_20200901_1226.')